<a href="https://colab.research.google.com/github/AdamFulton/thesis/blob/main/EEG_Emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [709]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,SubsetRandomSampler
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle as pickle
from sklearn.preprocessing import StandardScaler

In [710]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri May 20 12:14:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    39W / 250W |  11599MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [711]:
from google.colab import drive

drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [712]:
def read_eeg_signal_from_file(filename):
    x = pickle._Unpickler(open(filename, 'rb'))
    x.encoding = 'latin1'
    p = x.load()
    return p



In [713]:
files = []
for n in range(1, 33): 
    s = ''
    if n < 10:
        s += '0'
    s += str(n)
    files.append(s)
print(files)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']


In [714]:

labels = []
data = []

for i in files: 
    filename = "/content/gdrive/MyDrive/data_preprocessed_python/s" + i + ".dat"
    trial = read_eeg_signal_from_file(filename)
    labels.append(trial['labels'])
    data.append(trial['data'])

labels = np.array(labels)
labels = labels.flatten()
labels = labels.reshape(1280, 4)

data = np.array(data)
data = data.flatten()
data = data.reshape(1280, 40, 8064)

In [715]:
print("Labels: ", labels.shape)
print("Data: ", data.shape)

Labels:  (1280, 4)
Data:  (1280, 40, 8064)


In [716]:
df_label_ratings = pd.DataFrame({'Valence': labels[:,0], 'Arousal': labels[:,1]})
print(df_label_ratings.describe())

           Valence      Arousal
count  1280.000000  1280.000000
mean      5.254313     5.156711
std       2.130816     2.020499
min       1.000000     1.000000
25%       3.867500     3.762500
50%       5.040000     5.230000
75%       7.050000     6.950000
max       9.000000     9.000000


In [717]:
print(df_label_ratings.head(15))

    Valence  Arousal
0      7.71     7.60
1      8.10     7.31
2      8.58     7.54
3      4.94     6.01
4      6.96     3.92
5      8.27     3.92
6      7.44     3.73
7      7.32     2.55
8      4.04     3.29
9      1.99     4.86
10     2.99     2.36
11     2.71     2.77
12     1.95     3.12
13     4.18     2.24
14     3.17     8.08


In [718]:

df_hahv = df_label_ratings[(df_label_ratings['Valence'] >= np.median(labels[:,0])) & (df_label_ratings['Arousal'] >= np.median(labels[:,1]))]

df_lahv = df_label_ratings[(df_label_ratings['Valence'] >= np.median(labels[:,0])) & (df_label_ratings['Arousal'] < np.median(labels[:,1]))]

df_halv = df_label_ratings[(df_label_ratings['Valence'] < np.median(labels[:,0])) & (df_label_ratings['Arousal'] >= np.median(labels[:,1]))]

df_lalv = df_label_ratings[(df_label_ratings['Valence'] < np.median(labels[:,0])) & (df_label_ratings['Arousal'] < np.median(labels[:,1]))]

In [719]:
print("Positive Valence:", str(len(df_hahv) + len(df_lahv)))
print("Negative Valence:", str(len(df_halv) + len(df_lalv)))
print("High Arousal:", str(len(df_hahv) + len(df_halv)))
print("Low Arousal:", str(len(df_lahv) + len(df_lalv)))

Positive Valence: 680
Negative Valence: 600
High Arousal: 640
Low Arousal: 640


In [720]:
def positive_valence(trial):
    return 1 if labels[trial,0] >= np.median(labels[:,0]) else 0 

def high_arousal(trial):
    return 1 if labels[trial,1] >= np.median(labels[:,1]) else 0

In [721]:
labels_encoded = []
for i in range (len(labels)):
  labels_encoded.append([positive_valence(i), high_arousal(i)])

labels_encoded = np.reshape(labels_encoded, (1280, 2))
df_labels = pd.DataFrame(data=labels_encoded, columns=["Positive Valence", "High Arousal",])



In [722]:
df_valence = df_labels['Positive Valence']

df_arousal = df_labels['High Arousal']

df_arousal = df_arousal.to_numpy()

df_valence = df_valence.to_numpy()




In [723]:
eeg_channels = np.array(["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8", "PO4", "O2"])
peripheral_channels = np.array(["hEOG", "vEOG", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])


In [724]:
eeg_data = []
for i in range (len(data)):
    for j in range (len(eeg_channels)):
        eeg_data.append(data[i,j])
eeg_data = np.reshape(eeg_data, (len(data), len(eeg_channels), len(data[0,0])))
print(eeg_data.shape)

(1280, 32, 8064)


In [725]:
train_size = int(0.8 * len(eeg_data))
valid_size = len(eeg_data) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(eeg_data, [train_size, valid_size])
train_dataset = np.array(train_dataset)
valid_dataset = np.array(valid_dataset)
scaler = StandardScaler()




train = []
valid = []
print(len(train_dataset))
for i in range(valid_size,train_size):

    valid.append((train_dataset[i], df_valence[i]))
  

for i in range(0,len(valid_dataset)):

    train.append((valid_dataset[i],df_valence[i]))




print(train[0])

1024
(array([[  6.64561064,   1.9759769 , -10.09336038, ...,  -4.57772463,
        -10.84032221,   1.1085339 ],
       [ -5.81715954,  -5.61460809,   5.7950419 , ...,   6.59908672,
         13.79812842,   0.24619464],
       [  8.74115315,  -0.89235998,  -8.36698485, ...,  -0.14069965,
         -9.18724516,  -5.58496531],
       ...,
       [ -5.74765687,  -0.57955561,  -0.11323674, ...,  -3.57586788,
         -1.38762967,   3.55055799],
       [-10.3232662 ,  -4.68424519,  -6.79691768, ...,  -1.63490355,
          0.03873261,   9.83660886],
       [-15.25537631,  -6.07457524,  -4.84078595, ...,  -0.57490218,
          2.4679468 ,  13.10621666]]), 1)


In [ ]:
batch_size=22


train_dl = DataLoader(train,batch_size)


valid_dl = DataLoader(valid,batch_size)


len(train)

In [856]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv1d(inp, oup, 2, stride, 1, bias=False),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.25),
                nn.BatchNorm1d(oup),
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv1d(inp, inp, 2, stride, 1, groups=inp, bias=False),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.25),
                nn.BatchNorm1d(inp),
                
              
    
                nn.Conv1d(inp, oup, 2, 1, 0, bias=False),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.25),
                nn.BatchNorm1d(oup),
               
               
            )

        self.model = nn.Sequential(
            conv_bn( 32,  64, 2), 
            conv_dw( 64,  84, 1),
            conv_dw( 84, 104, 2),
            conv_dw(104, 124, 1),
            conv_dw(124, 144, 2),
            conv_dw(144, 164, 1),
            conv_dw(164, 184, 2),
            conv_dw(184, 204, 1),
            conv_dw(204, 256, 2),
            conv_dw(256, 312, 1),
            conv_dw(312, 412, 2),
            conv_dw(412, 512, 2),
            nn.AvgPool1d(60),
        )

        self.classifier = nn.Sequential(
        nn.Linear(512, 256),
      	nn.ReLU(),
        nn.Dropout(p=0.75),
        nn.Linear(256, 124),
        nn.ReLU(),
        nn.Dropout(p=0.75),
        nn.Linear(124, 64),
        nn.ReLU(),
        nn.Dropout(p=0.75),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Dropout(p=0.75),
        nn.Linear(32, 12),
        nn.Softmax(),
        nn.Linear(12,2)
        

        )
    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 512)
        x = self.classifier(x)
        return x

In [832]:
def get_default_device():

  if torch.cuda.is_available():
    return torch.device("cuda")

  else:
    return torch.device("cpu")

In [833]:
device = get_default_device();
device

device(type='cuda')

In [834]:
def to_device(data,device):

  if isinstance(data,(list,tuple)):
    return [to_device(x,device) for x in data]

  return data.to(device,non_blocking=True)

In [835]:
for data,labels in train_dl:
  print(data.shape)
  data = to_device(data,device)
  print(data.device)
  break

torch.Size([22, 32, 8064])
cuda:0


In [836]:
class DeviceDataLoader():
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
       
        return len(self.dl)

In [857]:
train_dl  = DeviceDataLoader(train_dl,device)
valid_dl = DeviceDataLoader(valid_dl,device)
model = Net()
to_device(model,device)

Net(
  (model): Sequential(
    (0): Sequential(
      (0): Conv1d(32, 64, kernel_size=(2,), stride=(2,), padding=(1,), bias=False)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.25, inplace=False)
      (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Conv1d(64, 64, kernel_size=(2,), stride=(1,), padding=(1,), groups=64, bias=False)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.25, inplace=False)
      (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): Conv1d(64, 84, kernel_size=(2,), stride=(1,), bias=False)
      (5): ReLU(inplace=True)
      (6): Dropout(p=0.25, inplace=False)
      (7): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Conv1d(84, 84, kernel_size=(2,), stride=(2,), padding=(1,), groups=84, bias=False)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.25, inplace=Fal

In [824]:
def accuracy(outputs,labels):
    _, preds = torch.max(outputs,dim=1)
    return torch.sum(preds == labels).item() / len(preds)

In [862]:

model = Net().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
min_valid_loss = np.inf

In [863]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for e in range(500):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    model.train()     
    for data, labels in train_dl:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        data = data.float()
        target = model(data)
        loss = loss_function(target,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_total += labels.shape[0]
        train_correct += torch.sum(labels == target.argmax(dim=-1))
    
    train_accuracy = train_correct / train_total
    valid_loss = 0.0
    valid_correct = 0
    valid_total = 0
    model.eval() 
    with torch.no_grad():   
      for data, labels in valid_dl:
          if torch.cuda.is_available():
              data, labels = data.cuda(), labels.cuda()
          data = data.float()
          target = model(data)
          loss = loss_function(target,labels)
          valid_loss = loss.item() * data.size(0)
          valid_loss += loss.item()
          valid_total += labels.shape[0]
          valid_correct += torch.sum(labels == target.argmax(dim=-1))
      
    valid_accuracy = valid_correct / valid_total

    print(f'Epoch {e+1} \t\t Train Accuracy: {train_accuracy.item()} \t\t Training Loss: {train_loss / len(train_dl)} \t\t Valid Accuracy: {valid_accuracy.item()} \t\t Validation Loss: {valid_loss / len(valid_dl)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_model.pth')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 1 		 Train Accuracy: 0.43359375 		 Training Loss: 0.7152092158794403 		 Valid Accuracy: 0.4908854365348816 		 Validation Loss: 0.4606847405433655
Validation Loss Decreased(inf--->16.123966) 	 Saving The Model
Epoch 2 		 Train Accuracy: 0.43359375 		 Training Loss: 0.7110955317815145 		 Valid Accuracy: 0.4908854365348816 		 Validation Loss: 0.4528971791267395
Validation Loss Decreased(16.123966--->15.851401) 	 Saving The Model
Epoch 3 		 Train Accuracy: 0.43359375 		 Training Loss: 0.7069870481888453 		 Valid Accuracy: 0.4908854365348816 		 Validation Loss: 0.4450886964797974
Validation Loss Decreased(15.851401--->15.578104) 	 Saving The Model
Epoch 4 		 Train Accuracy: 0.43359375 		 Training Loss: 0.703812782963117 		 Valid Accuracy: 0.4908854365348816 		 Validation Loss: 0.436947762966156
Validation Loss Decreased(15.578104--->15.293172) 	 Saving The Model
Epoch 5 		 Train Accuracy: 0.43359375 		 Training Loss: 0.7006748815377554 		 Valid Accuracy: 0.4908854365348816 		 Validati

KeyboardInterrupt: ignored